<a href="https://colab.research.google.com/github/leandroclv/polars_alura/blob/main/introducao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Carregando o conjunto de dados

A partir do conjunto de dados CSV que criamos antes:

df = pl.read_csv("transactions.csv")

print(df.head())

##Tratamento de valores nulos

- fill_null(): Substitua os valores nulos por um valor específico.
- drop_nulls(): Tira as linhas que têm valores nulos.
- is_null() / is_not_null(): Verificações booleanas.

##Preencher valores ausentes
Valores ausentes podem causar problemas na análise posterior e na visualização dos dados. Você vai precisar preencher os valores que faltam pra garantir que tudo corra bem.

Veja como você pode preencher os valores que estão faltando:

- Imagine 'amount' has missing values, then replace with 0

df_filled = df.with_columns(
    pl.col("amount").fill_null(0)
)

##Eliminar valores nulos
Os nulos podem causar erros se não forem tratados. Veja como eliminá-los:

df_no_nulls = df.drop_nulls()

##Selecionar colunas
Quando você usa o Polars, uma coisa comum é escolher as colunas que você quer usar e manter.

- Select only transaction_id and amount

df.select(["transaction_id", "amount"])

##Filtrar linhas
Você também pode filtrar linhas com base nas condições que definir, parecido com o pandas.

- Get only high-value transactions above $1,000

high_value = df.filter(pl.col("amount") > 1000)

print(high_value)


##Aplicar expressões
Aplicar expressões no Polars envolve usar o objeto polars.Expr em vários contextos para fazer transformações de dados.

Você pode criá-los usando pl.col() ou pl.lit().

- Add a 10% discount column to simulate promotional pricing

df.select([
    pl.col("transaction_id"),
    pl.col("amount"),
    (pl.col("amount") * 0.9).alias("discounted_amount")
])

##Agregados
A gente pode ver como os clientes gastam usando o group_by.

- Total and average spend per customer

agg_df = df.group_by("customer_id").agg([
    pl.sum("amount").alias("total_spent"),
    pl.mean("amount").alias("avg_transaction")
])

print(agg_df)

##Aqui vai um exemplo do modo preguiçoso:

import polars as pl

- Load dataset in lazy mode
lazy_df = pl.scan_csv("transactions.csv")

- Build a transformation pipeline

pipeline = (
    lazy_df
    .filter(pl.col("amount") > 1000)                # step 1: filter expensive transactions
    .group_by("customer_id")                         # step 2: group by customer
    .agg(pl.sum("amount").alias("total_spent"))     # step 3: aggregate
)

- Nothing has run yet, computation happens only on collect()

result = pipeline.collect()
print(result)

Junções suportadas
O Polars aceita todos os principais tipos de junção:

Inner Join: Guarde só as linhas que combinam.
União à esquerda/direita: Mantenha todas as linhas de um lado.
: Mantenha todas as linhas, preenchendo as lacunas com nulos.
Semi Join: Mantenha as linhas da esquerda que têm uma correspondência à direita.
Anti Join: Mantenha as linhas da esquerda que não têm correspondência à direita.
: Produto cartesiano das duas tabelas.
Exemplo: Juntando transações com metadados do cliente

# Create customer metadata DataFrame
customers = pl.DataFrame({
    "customer_id": [1, 2, 3, 4, 5],
    "customer_name": ["Alice", "Bob", "Charlie", "David", "Eva"]
})

# Inner join on customer_id
df_joined = df.join(customers, on="customer_id", how="inner")
print(df_joined.head())

In [ ]:
import polars as pl

In [ ]:
restaurantes = pl.read_csv('//content/drive/MyDrive/Colab Notebooks/Análise de Dados com Polars: O Novo Padrão do Python - Alura/restaurantes.csv', encoding='utf-8')

In [ ]:
print(restaurantes)

shape: (51_717, 13)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ name       ┆ online_ord ┆ book_tabl ┆ rate   ┆ … ┆ reviews_l ┆ menu_item ┆ listed_in ┆ listed_in │
│ ---        ┆ er         ┆ e         ┆ ---    ┆   ┆ ist       ┆ ---       ┆ (type)    ┆ (city)    │
│ str        ┆ ---        ┆ ---       ┆ str    ┆   ┆ ---       ┆ str       ┆ ---       ┆ ---       │
│            ┆ str        ┆ str       ┆        ┆   ┆ str       ┆           ┆ str       ┆ str       │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Jalsa      ┆ Yes        ┆ Yes       ┆ 4.1/5  ┆ … ┆ [('Rated  ┆ []        ┆ Buffet    ┆ Banashank │
│            ┆            ┆           ┆        ┆   ┆ 4.0',     ┆           ┆           ┆ ari       │
│            ┆            ┆           ┆        ┆   ┆ 'RATED\n  ┆           ┆           ┆           │
│            ┆            ┆           ┆        ┆   ┆ A bea…    ┆       

In [ ]:
restaurantes.glimpse()

Rows: 51717
Columns: 13
$ name                        <str> 'Jalsa', 'Spice Elephant', 'San Churro Cafe', 'Addhuri Udupi Bhojana', 'Grand Village', 'Timepass Dinner', 'Rosewood International Hotel - Bar & Restaurant', 'Onesta', 'Penthouse Cafe', 'Smacznego'
$ online_order                <str> 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes'
$ book_table                  <str> 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No'
$ rate                        <str> '4.1/5', '4.1/5', '3.8/5', '3.7/5', '3.8/5', '3.8/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5'
$ votes                       <i64> 775, 787, 918, 88, 166, 286, 8, 2556, 324, 504
$ rest_type                   <str> 'Casual Dining', 'Casual Dining', 'Cafe, Casual Dining', 'Quick Bites', 'Casual Dining', 'Casual Dining', 'Casual Dining', 'Casual Dining, Cafe', 'Cafe', 'Cafe'
$ dish_liked                  <str> 'Pasta, Lunch Buffet, Masala Papad, Paneer Lajawab, Tomato Shorba, Dum Biryani, Sweet Corn Soup', 'Mom

In [ ]:
#Convertendo valores numéricos

#Substituindo virgulas nos itens numéricos
restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',', '')
)

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,str,i64,str,str,str,str,str,str,str,str
"""Jalsa""","""Yes""","""Yes""","""4.1/5""",775,"""Casual Dining""","""Pasta, Lunch Buffet, Masala Pa…","""North Indian, Mughlai, Chinese""","""800""","""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""","""4.1/5""",787,"""Casual Dining""","""Momos, Lunch Buffet, Chocolate…","""Chinese, North Indian, Thai""","""800""","""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""","""3.8/5""",918,"""Cafe, Casual Dining""","""Churros, Cannelloni, Minestron…","""Cafe, Mexican, Italian""","""800""","""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""","""3.7/5""",88,"""Quick Bites""","""Masala Dosa""","""South Indian, North Indian""","""300""","""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""","""3.8/5""",166,"""Casual Dining""","""Panipuri, Gol Gappe""","""North Indian, Rajasthani""","""600""","""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""","""3.6 /5""",27,"""Bar""",null,"""Continental""","""1500""","""[('Rated 5.0', ""RATED\n Food …","""[]""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""","""600""","""[]""","""[]""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""","""2000""","""[]""","""[]""","""Pubs and bars""","""Whitefield"""


In [ ]:
#Convertendo strings em floats
restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',','')
    .cast(pl.Float64)
)

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,str,i64,str,str,str,f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""","""4.1/5""",775,"""Casual Dining""","""Pasta, Lunch Buffet, Masala Pa…","""North Indian, Mughlai, Chinese""",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""","""4.1/5""",787,"""Casual Dining""","""Momos, Lunch Buffet, Chocolate…","""Chinese, North Indian, Thai""",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""","""3.8/5""",918,"""Cafe, Casual Dining""","""Churros, Cannelloni, Minestron…","""Cafe, Mexican, Italian""",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""","""3.7/5""",88,"""Quick Bites""","""Masala Dosa""","""South Indian, North Indian""",300.0,"""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""","""3.8/5""",166,"""Casual Dining""","""Panipuri, Gol Gappe""","""North Indian, Rajasthani""",600.0,"""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""","""3.6 /5""",27,"""Bar""",null,"""Continental""",1500.0,"""[('Rated 5.0', ""RATED\n Food …","""[]""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""",600.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""",2000.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""


In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',','')
    .cast(pl.Float64)
)

In [ ]:
restaurantes

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,str,i64,str,str,str,f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""","""4.1/5""",775,"""Casual Dining""","""Pasta, Lunch Buffet, Masala Pa…","""North Indian, Mughlai, Chinese""",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""","""4.1/5""",787,"""Casual Dining""","""Momos, Lunch Buffet, Chocolate…","""Chinese, North Indian, Thai""",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""","""3.8/5""",918,"""Cafe, Casual Dining""","""Churros, Cannelloni, Minestron…","""Cafe, Mexican, Italian""",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""","""3.7/5""",88,"""Quick Bites""","""Masala Dosa""","""South Indian, North Indian""",300.0,"""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""","""3.8/5""",166,"""Casual Dining""","""Panipuri, Gol Gappe""","""North Indian, Rajasthani""",600.0,"""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""","""3.6 /5""",27,"""Bar""",null,"""Continental""",1500.0,"""[('Rated 5.0', ""RATED\n Food …","""[]""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""",600.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""",2000.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""


In [ ]:
restaurantes.select(
    pl.col('approx_cost(for two people)').max().alias('max'),
    pl.col('approx_cost(for two people)').min().alias('min'),
    pl.col('approx_cost(for two people)').mean().round(2).alias('média')
)

max,min,média
f64,f64,f64
6000.0,40.0,555.43


In [ ]:
restaurantes.glimpse()

Rows: 51717
Columns: 13
$ name                        <str> 'Jalsa', 'Spice Elephant', 'San Churro Cafe', 'Addhuri Udupi Bhojana', 'Grand Village', 'Timepass Dinner', 'Rosewood International Hotel - Bar & Restaurant', 'Onesta', 'Penthouse Cafe', 'Smacznego'
$ online_order                <str> 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes'
$ book_table                  <str> 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No'
$ rate                        <str> '4.1/5', '4.1/5', '3.8/5', '3.7/5', '3.8/5', '3.8/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5'
$ votes                       <i64> 775, 787, 918, 88, 166, 286, 8, 2556, 324, 504
$ rest_type                   <str> 'Casual Dining', 'Casual Dining', 'Cafe, Casual Dining', 'Quick Bites', 'Casual Dining', 'Casual Dining', 'Casual Dining', 'Casual Dining, Cafe', 'Cafe', 'Cafe'
$ dish_liked                  <str> 'Pasta, Lunch Buffet, Masala Papad, Paneer Lajawab, Tomato Shorba, Dum Biryani, Sweet Corn Soup', 'Mom


Nesta aula, aprendemos:

    - A usar a biblioteca Polars para manipulação eficiente de grandes volumes de dados em Python.
    - A realizar a leitura de arquivos CSV com a função 'read_csv', destacando a importância do encoding correto.
    - A explorar dados carregados utilizando o método 'glimpse' para ter uma visão geral da estrutura dos dados.
    - A converter tipos de dados, transformando strings em valores numéricos para análises precisas.
    - A aplicar transformações em colunas com o método 'with_columns' e 'str.replace' para manipulação de strings.
    - A usar o método 'cast' para conversão de tipos numa coluna, de string para numérico.
    - A aplicar funções estatísticas como max, min e mean com o método 'select'.
    - A arredondar valores numéricos com o método 'round' para ajustes de casas decimais.

In [ ]:
#Regex
#^([\d\.]+)

Regex, ou Expressões Regulares, é uma ferramenta poderosa usada para encontrar, extrair e manipular padrões de texto em strings. Ela é amplamente utilizada em linguagens de programação, bancos de dados e editores de texto para realizar buscas e substituições eficientes.

Como funciona?

A Regex funciona através de um conjunto de caracteres especiais que definem padrões. Por exemplo:

    . (ponto) representa qualquer caractere.
    * (asterisco) significa que o caractere anterior pode aparecer zero ou mais vezes.
    + (mais) indica que o caractere anterior deve aparecer pelo menos uma vez.
    ? (interrogação) faz com que o caractere anterior seja opcional.
    \d corresponde a qualquer dígito (0-9).
    \w corresponde a qualquer letra ou número.
    ^ indica o início da linha.
    $ indica o fim da linha.

Exemplos de aplicação

1 - Validar um e-mail: um padrão básico para validar e-mails pode ser:

^[\w.-]+@[\w.-]+\.\w+$

Este padrão verifica se um e-mail tem formato válido com nome, @, domínio e extensão.

2 - Encontrar números em um texto: se quisermos localizar todos os números em um texto, podemos usar:

\d+

Isso irá capturar sequências de números, como "123" ou "2025".

3 - Substituir espaços extras por um único espaço: para remover espaços desnecessários em um texto:

\s+

Isso substitui múltiplos espaços por um único espaço.

In [ ]:
restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
)

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,str,i64,str,str,str,f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""","""4.1""",775,"""Casual Dining""","""Pasta, Lunch Buffet, Masala Pa…","""North Indian, Mughlai, Chinese""",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""","""4.1""",787,"""Casual Dining""","""Momos, Lunch Buffet, Chocolate…","""Chinese, North Indian, Thai""",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""","""3.8""",918,"""Cafe, Casual Dining""","""Churros, Cannelloni, Minestron…","""Cafe, Mexican, Italian""",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""","""3.7""",88,"""Quick Bites""","""Masala Dosa""","""South Indian, North Indian""",300.0,"""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""","""3.8""",166,"""Casual Dining""","""Panipuri, Gol Gappe""","""North Indian, Rajasthani""",600.0,"""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""","""3.6""",27,"""Bar""",null,"""Continental""",1500.0,"""[('Rated 5.0', ""RATED\n Food …","""[]""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""",600.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""",2000.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""


In [ ]:
#Transformou strings em números
restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
    .cast(pl.Float64)
)

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,f64,i64,str,str,str,f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""",4.1,775,"""Casual Dining""","""Pasta, Lunch Buffet, Masala Pa…","""North Indian, Mughlai, Chinese""",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""",4.1,787,"""Casual Dining""","""Momos, Lunch Buffet, Chocolate…","""Chinese, North Indian, Thai""",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""",3.8,918,"""Cafe, Casual Dining""","""Churros, Cannelloni, Minestron…","""Cafe, Mexican, Italian""",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""",3.7,88,"""Quick Bites""","""Masala Dosa""","""South Indian, North Indian""",300.0,"""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""",3.8,166,"""Casual Dining""","""Panipuri, Gol Gappe""","""North Indian, Rajasthani""",600.0,"""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""",3.6,27,"""Bar""",null,"""Continental""",1500.0,"""[('Rated 5.0', ""RATED\n Food …","""[]""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""",600.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"""Finger Food""",2000.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""


In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
    .cast(pl.Float64)
)

In [ ]:
restaurantes.head()

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,f64,i64,str,str,str,f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""",4.1,775,"""Casual Dining""","""Pasta, Lunch Buffet, Masala Pa…","""North Indian, Mughlai, Chinese""",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""",4.1,787,"""Casual Dining""","""Momos, Lunch Buffet, Chocolate…","""Chinese, North Indian, Thai""",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""",3.8,918,"""Cafe, Casual Dining""","""Churros, Cannelloni, Minestron…","""Cafe, Mexican, Italian""",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""",3.7,88,"""Quick Bites""","""Masala Dosa""","""South Indian, North Indian""",300.0,"""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""",3.8,166,"""Casual Dining""","""Panipuri, Gol Gappe""","""North Indian, Rajasthani""",600.0,"""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""


In [ ]:
restaurantes.select(
    pl.col('rate').max().alias('max'),
    pl.col('rate').min().alias('min')
)

max,min
f64,f64
4.9,1.8


In [ ]:
#Convertendo texto para listas
restaurantes.with_columns(
    pl.col('dish_liked')
    .cast(pl.List(pl.String)),
    pl.col('cuisines')
    .cast(pl.List(pl.String))

)

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,f64,i64,str,list[str],list[str],f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""",4.1,775,"""Casual Dining""","[""Pasta, Lunch Buffet, Masala Papad, Paneer Lajawab, Tomato Shorba, Dum Biryani, Sweet Corn Soup""]","[""North Indian, Mughlai, Chinese""]",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""",4.1,787,"""Casual Dining""","[""Momos, Lunch Buffet, Chocolate Nirvana, Thai Green Curry, Paneer Tikka, Dum Biryani, Chicken Biryani""]","[""Chinese, North Indian, Thai""]",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""",3.8,918,"""Cafe, Casual Dining""","[""Churros, Cannelloni, Minestrone Soup, Hot Chocolate, Pink Sauce Pasta, Salsa, Veg Supreme Pizza""]","[""Cafe, Mexican, Italian""]",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""",3.7,88,"""Quick Bites""","[""Masala Dosa""]","[""South Indian, North Indian""]",300.0,"""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""",3.8,166,"""Casual Dining""","[""Panipuri, Gol Gappe""]","[""North Indian, Rajasthani""]",600.0,"""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""",3.6,27,"""Bar""",null,"[""Continental""]",1500.0,"""[('Rated 5.0', ""RATED\n Food …","""[]""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"[""Finger Food""]",600.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"[""Finger Food""]",2000.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""


In [ ]:
restaurantes.with_columns(
    pl.col('dish_liked')
    .str.split(','),
    pl.col('cuisines')
    .str.split(',')
)

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,f64,i64,str,list[str],list[str],f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""",4.1,775,"""Casual Dining""","[""Pasta"", "" Lunch Buffet"", … "" Sweet Corn Soup""]","[""North Indian"", "" Mughlai"", "" Chinese""]",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""",4.1,787,"""Casual Dining""","[""Momos"", "" Lunch Buffet"", … "" Chicken Biryani""]","[""Chinese"", "" North Indian"", "" Thai""]",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""",3.8,918,"""Cafe, Casual Dining""","[""Churros"", "" Cannelloni"", … "" Veg Supreme Pizza""]","[""Cafe"", "" Mexican"", "" Italian""]",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""",3.7,88,"""Quick Bites""","[""Masala Dosa""]","[""South Indian"", "" North Indian""]",300.0,"""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""",3.8,166,"""Casual Dining""","[""Panipuri"", "" Gol Gappe""]","[""North Indian"", "" Rajasthani""]",600.0,"""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""",3.6,27,"""Bar""",null,"[""Continental""]",1500.0,"""[('Rated 5.0', ""RATED\n Food …","""[]""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"[""Finger Food""]",600.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"[""Finger Food""]",2000.0,"""[]""","""[]""","""Pubs and bars""","""Whitefield"""


In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('dish_liked')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars()),
    pl.col('cuisines')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars())
)

In [ ]:
restaurantes.head()

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,f64,i64,str,list[str],list[str],f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""",4.1,775,"""Casual Dining""","[""Pasta"", ""Lunch Buffet"", … ""Sweet Corn Soup""]","[""North Indian"", ""Mughlai"", ""Chinese""]",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""[]""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""",4.1,787,"""Casual Dining""","[""Momos"", ""Lunch Buffet"", … ""Chicken Biryani""]","[""Chinese"", ""North Indian"", ""Thai""]",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""[]""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""",3.8,918,"""Cafe, Casual Dining""","[""Churros"", ""Cannelloni"", … ""Veg Supreme Pizza""]","[""Cafe"", ""Mexican"", ""Italian""]",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""[]""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""",3.7,88,"""Quick Bites""","[""Masala Dosa""]","[""South Indian"", ""North Indian""]",300.0,"""[('Rated 4.0', ""RATED\n Great…","""[]""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""",3.8,166,"""Casual Dining""","[""Panipuri"", ""Gol Gappe""]","[""North Indian"", ""Rajasthani""]",600.0,"""[('Rated 4.0', 'RATED\n Very …","""[]""","""Buffet""","""Banashankari"""


In [ ]:
restaurantes.select(
    pl.col('menu_item').unique()
).item(0,0)

'[\'Brownie Fudge\', \'Hot Chocolate Fudge( Regular )\', \'Hot Chocolate Fudge( Jumbo )\', \'Choco Brownie Fudge\', \'Red Velvet Choco Sin\', \'Dry Fruit Special\', \'Seven Wonders\', \'Oreo Shake\', \'Dbc Shake\', \'Berry Colate\', \'Brownie Fudge\', \'Hot Chocolate Fudge( Junior )\', \'Hot Chocolate Fudge( Regular )\', \'Hot Chocolate Fudge( Jumbo )\', \'Choco Brownie Fudge\', \'Red Velvet Choco Sin\', \'Mini Mocha\', \'Deep Purple\', \'Dry Fruit Special\', \'Ice Cream Pizza Slice\', \'Family Pizza\', \'Seven Wonders\', \'Lychee Elite\', \'Kesar Kulfi\', \'Movie Matinee\', \'Butter Scotch Blast\', \'Mango Berry\', \'Banana Almond Scotch\', \'Fruit Zest\', \'Lychee Sundae\', \'Black Magic\', \'Wild Strawberry\', \'Chocolate\', \'Cold Coffee\', \'Oreo Shake\', \'Caramel-vanilla\', \'Banana-mango\', \'Red Velvet Oreo\', \'Dbc Shake\', \'Strawberry\', \'Rich Chocolate\', \'Alphonso Mango\', \'Fresh Pineapple\', \'Butterscotch\', "Fig O\' Honey", \'Vanilla Ice Cream\', \'Strawberry Ice Cr

In [ ]:
restaurantes.with_columns(
    pl.col('menu_item')
    .str.replace('\[', '')
    .str.replace('\]', '')
)

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\]'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\]'
/tmp/ipython-input-3055492535.py:3: SyntaxWarning: invalid escape sequence '\['
  .str.replace('\[', '')
/tmp/ipython-input-3055492535.py:4: SyntaxWarning: invalid escape sequence '\]'
  .str.replace('\]', '')


name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,f64,i64,str,list[str],list[str],f64,str,str,str,str
"""Jalsa""","""Yes""","""Yes""",4.1,775,"""Casual Dining""","[""Pasta"", ""Lunch Buffet"", … ""Sweet Corn Soup""]","[""North Indian"", ""Mughlai"", ""Chinese""]",800.0,"""[('Rated 4.0', 'RATED\n A bea…","""""","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""",4.1,787,"""Casual Dining""","[""Momos"", ""Lunch Buffet"", … ""Chicken Biryani""]","[""Chinese"", ""North Indian"", ""Thai""]",800.0,"""[('Rated 4.0', 'RATED\n Had b…","""""","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""",3.8,918,"""Cafe, Casual Dining""","[""Churros"", ""Cannelloni"", … ""Veg Supreme Pizza""]","[""Cafe"", ""Mexican"", ""Italian""]",800.0,"""[('Rated 3.0', ""RATED\n Ambie…","""""","""Buffet""","""Banashankari"""
"""Addhuri Udupi Bhojana""","""No""","""No""",3.7,88,"""Quick Bites""","[""Masala Dosa""]","[""South Indian"", ""North Indian""]",300.0,"""[('Rated 4.0', ""RATED\n Great…","""""","""Buffet""","""Banashankari"""
"""Grand Village""","""No""","""No""",3.8,166,"""Casual Dining""","[""Panipuri"", ""Gol Gappe""]","[""North Indian"", ""Rajasthani""]",600.0,"""[('Rated 4.0', 'RATED\n Very …","""""","""Buffet""","""Banashankari"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Best Brews - Four Points by Sh…","""No""","""No""",3.6,27,"""Bar""",null,"[""Continental""]",1500.0,"""[('Rated 5.0', ""RATED\n Food …","""""","""Pubs and bars""","""Whitefield"""
"""Vinod Bar And Restaurant""","""No""","""No""",null,0,"""Bar""",null,"[""Finger Food""]",600.0,"""[]""","""""","""Pubs and bars""","""Whitefield"""
"""Plunge - Sheraton Grand Bengal…","""No""","""No""",null,0,"""Bar""",null,"[""Finger Food""]",2000.0,"""[]""","""""","""Pubs and bars""","""Whitefield"""


In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('menu_item')
    .str.replace('\[', '')
    .str.replace('\]', '')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars())
)

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\]'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\]'
/tmp/ipython-input-2628076539.py:3: SyntaxWarning: invalid escape sequence '\['
  .str.replace('\[', '')
/tmp/ipython-input-2628076539.py:4: SyntaxWarning: invalid escape sequence '\]'
  .str.replace('\]', '')


In [ ]:
restaurantes.filter(
    pl.col('menu_item').list.len() > 1
).select(
    pl.col('menu_item')
).item(0,0).to_list()

["'Chocolate Fantasy (Pack Of 5)'",
 "'Pan Cake (Pack Of 6)'",
 "'Gulab Jamun (Pack Of 10)'",
 "'Gulkand Shot (Pack Of 5)'",
 "'Chocolate Decadence (Pack of 2)'",
 "'CheeseCake (Pack Of 2)'",
 "'Red Velvet Slice Cake (Pack of 2)'",
 "'Red Velvet Slice Cake & Cheese Cake (Pack of 2)'",
 "'Red Velvet Slice Cake & Chocolate Decadence Cake (Pack of 2)'",
 "'Hazelnut Brownie (Pack of 2)'",
 "'Moments'",
 "'Red Velvet Cake With Butter Cream Frosting (750 Gm)'",
 "'Red Velvet Slice Cake (Pack of 2)'",
 "'Red Velvet Slice Cake & Cheese Cake (Pack of 2)'",
 "'Red Velvet Slice Cake & Chocolate Decadence Cake (Pack of 2)'",
 "'Red Velvet Slice Cake (Pack of 1)'",
 "'Valentine Red Velvet Jar'",
 "'Valentine Chocolate Jar'",
 "'Valentines Jar Combo'",
 "'Pink Guava 500 ML'",
 "'Oreo Vanilla 500 ML'",
 "'Cookie Crumble 500 ML'",
 "'Chocolate Fantasy'",
 "'Gulkand-E-Bahar'",
 "'Pan Cake'",
 "'Hazelnut Brownie (Pack Of 1)'",
 "'Gulab Jamun (Pack Of 2)'",
 "'Plum Cake'",
 "'Red Velvet Cake With Butter 

In [ ]:
restaurantes.select(
    pl.col('reviews_list')
).item(0,0)

'[(\'Rated 4.0\', \'RATED\\n  A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.\'), (\'Rated 4.0\', \'RATED\\n  I was here for dinner with my family on a weekday. The restaurant was completely empty. Ambience is good with some good old hindi music. Seating arrangement are good too. We ordered masala papad, panner and baby corn starters, lemon and corrionder soup, butter roti, olive and chilli paratha. Food was fresh and good, service is good too. Good for family hangout.\\nCheers\'), (\'Rated 2.0\', \'RATED\\n  Its a restaurant near to Banashankari BDA. Me along with few of my office friends visited to have buffet but unfortunately they only provide veg buffet. On inquiring they said this plac

In [ ]:
restaurantes.with_columns(
    pl.col('reviews_list')
    .str.replace('\[','')
    .str.replace('\]','')
    .str.replace('\(','')
).select(
    pl.col('reviews_list')
).item(0,0)

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\]'
<>:5: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\]'
<>:5: SyntaxWarning: invalid escape sequence '\('
/tmp/ipython-input-442479560.py:3: SyntaxWarning: invalid escape sequence '\['
  .str.replace('\[','')
/tmp/ipython-input-442479560.py:4: SyntaxWarning: invalid escape sequence '\]'
  .str.replace('\]','')
/tmp/ipython-input-442479560.py:5: SyntaxWarning: invalid escape sequence '\('
  .str.replace('\(','')


'\'Rated 4.0\', \'RATED\\n  A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.\'), (\'Rated 4.0\', \'RATED\\n  I was here for dinner with my family on a weekday. The restaurant was completely empty. Ambience is good with some good old hindi music. Seating arrangement are good too. We ordered masala papad, panner and baby corn starters, lemon and corrionder soup, butter roti, olive and chilli paratha. Food was fresh and good, service is good too. Good for family hangout.\\nCheers\'), (\'Rated 2.0\', \'RATED\\n  Its a restaurant near to Banashankari BDA. Me along with few of my office friends visited to have buffet but unfortunately they only provide veg buffet. On inquiring they said this place 

In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('reviews_list')
    .str.replace('\[','')
    .str.replace('\]','')
    .str.replace('\(','')
    .str.split('), (')
)

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\]'
<>:5: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\]'
<>:5: SyntaxWarning: invalid escape sequence '\('
/tmp/ipython-input-3109954863.py:3: SyntaxWarning: invalid escape sequence '\['
  .str.replace('\[','')
/tmp/ipython-input-3109954863.py:4: SyntaxWarning: invalid escape sequence '\]'
  .str.replace('\]','')
/tmp/ipython-input-3109954863.py:5: SyntaxWarning: invalid escape sequence '\('
  .str.replace('\(','')


In [ ]:
restaurantes.head()

name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
str,str,str,f64,i64,str,list[str],list[str],f64,list[str],list[str],str,str
"""Jalsa""","""Yes""","""Yes""",4.1,775,"""Casual Dining""","[""Pasta"", ""Lunch Buffet"", … ""Sweet Corn Soup""]","[""North Indian"", ""Mughlai"", ""Chinese""]",800.0,"[""'Rated 4.0', 'RATED\n A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.'"", ""'Rated 4.0', 'RATED\n I was here for dinner with my family on a weekday. The restaurant was completely empty. Ambience is good with some good old hindi music. Seating arrangement are good too. We ordered masala papad, panner and baby corn starters, lemon and corrionder soup, butter roti, olive and chilli paratha. Food was fresh and good, service is good too. Good for family hangout.\nCheers'"", … ""'Rated 4.0', 'RATED\n The place is nice and comfortable. Food wise all jalea outlets maintain a good standard. The soya chaap was a standout dish. Clearly one of trademark dish as per me and a must try.\n\nThe only concern is the parking. It very congested and limited to just 5cars. The basement parking is very steep and makes it cumbersome')""]","[""""]","""Buffet""","""Banashankari"""
"""Spice Elephant""","""Yes""","""No""",4.1,787,"""Casual Dining""","[""Momos"", ""Lunch Buffet"", … ""Chicken Biryani""]","[""Chinese"", ""North Indian"", ""Thai""]",800.0,"[""'Rated 4.0', 'RATED\n Had been here for dinner with family. Turned out to be a good choose suitable for all ages of people. Can try this place. We liked the most was their starters. Service is good. Prices are affordable. Will recommend this restaurant for early dinner. The place is little noisy.'"", ""'Rated 3.0', 'RATED\n The ambience is really nice, staff is courteous. The price is pretty high for the quantity, but overall the experience was fine. The quality of food is nice but nothing extraordinary. They also have buffet(only veg)'"", … ""'Rated 4.0', 'RATED\n Zomato gold partner at this price. It was insane. They have really nice food. small place with very courteous staff and very cheap food for this ambience. Cost of soups is 80-100. Starters from 150-250. Main course 200-300. Cost for two was 800 for us.')""]","[""""]","""Buffet""","""Banashankari"""
"""San Churro Cafe""","""Yes""","""No""",3.8,918,"""Cafe, Casual Dining""","[""Churros"", ""Cannelloni"", … ""Veg Supreme Pizza""]","[""Cafe"", ""Mexican"", ""Italian""]",800.0,"[""'Rated 3.0', ""RATED\n Ambience is not that good enough and it's not a pocket friendly cafe and the quantity is not that good and desserts are too good enough ??.."""", ""'Rated 3.0', ""RATED\n \nWent there for a quick bite with friends.\nThe ambience had more of corporate feel. I would say it was unique.\nTried nachos, pasta churros and lasagne.\n\nNachos were pathetic.( Seriously don't order)\nPasta was okayish.\nLasagne was good.\nNutella churros were the best.\nOverall an okayish experience!\nPeace ??"""", … ""'Rated 2.0', ""RATED\n 2nd time I have visited this place and I am surprised to see how it has declined. Churros was hard, pizza was pathetic, loaded nachos and masala Garlic bread were topped with cheese with MAYO! Yes mayonnese.. Soup has the bread crumbs in almost melted state, mocktails were average and our only Savior was pasta and caramel pudding... This place has high potential but let down by dull ambience, pretty slow and inattentive staff. Buffet price was listed as 399 but was charged 499, We still went ahead and took the deal to be very disappointed!! Most dishes were freshly cooked hence going with some starts else would rate them lower. I did not find a singl

aprendemos:

    - A utilizar expressões regulares com a função extract do Polars para capturar padrões em cadeias de texto.
    - A converter valores de string para formato numérico usando o método cast.
    - A determinar valores máximo e mínimo de uma coluna numérica utilizando max e min.
    - A transformar colunas de strings em listas com str.split e limpar dados com list.eval e strip_chars.
    - A identificar valores distintos com o método unique e acessar elementos específicos usando item.
    - A manipular e limpar dados textuais com replace e split, preparando-os para análise.
    - A filtrar elementos em um DataFrame com filter e realizar transformações.

In [ ]:
restaurantes.select(
    [pl.col('online_order').value_counts()]
)

online_order
struct[2]
"{""Yes"",30444}"
"{""No"",21273}"


In [ ]:
restaurantes.select(
    [pl.col('online_order').value_counts()]
).unnest('online_order')

online_order,count
str,u32
"""No""",21273
"""Yes""",30444


In [ ]:
total_registros = restaurantes.height
total_registros

51717

In [ ]:
restaurantes.select([
    pl.col('online_order').value_counts()
]).unnest('online_order').with_columns(
    (pl.col('count')/total_registros * 100).round(2)
)

online_order,count
str,f64
"""No""",41.13
"""Yes""",58.87


In [ ]:
restaurantes.select([
    pl.col('book_table').value_counts()
]).unnest('book_table').with_columns(
    (pl.col('count')/total_registros * 100).round(2)
)

book_table,count
str,f64
"""No""",87.53
"""Yes""",12.47


In [ ]:
restaurantes.select([
    pl.col('rest_type').n_unique(),
    pl.col('listed_in(type)').n_unique()
])

rest_type,listed_in(type)
u32,u32
94,7


In [ ]:
restaurantes.select([
    pl.col('listed_in(type)').value_counts()
]).unnest('listed_in(type)').sort('count', descending=True)

listed_in(type),count
str,u32
"""Delivery""",25942
"""Dine-out""",17779
"""Desserts""",3593
"""Cafes""",1723
"""Drinks & nightlife""",1101
"""Buffet""",882
"""Pubs and bars""",697


In [ ]:
restaurantes.select([
    pl.col('rest_type').value_counts()
]).unnest('rest_type').sort('count', descending=True).head(10)

rest_type,count
str,u32
"""Quick Bites""",19132
"""Casual Dining""",10330
"""Cafe""",3732
"""Delivery""",2604
"""Dessert Parlor""",2263
"""Takeaway, Delivery""",2037
"""Casual Dining, Bar""",1154
"""Bakery""",1141
"""Beverage Shop""",867
